In [1]:
!pip install surprise

     |████████████████████████████████| 11.8MB 210kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1670951 sha256=fbcd7b88fc26dab9ccbfb718ef607e2db931f509102f60541399f2129010d0f5
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [2]:
!pip install implicit

     |████████████████████████████████| 1.1MB 4.5MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp36-cp36m-linux_x86_64.whl size=3419504 sha256=7146640513e3a8e21611250bdec3cf7fecbf4528cf1eca4a17a965ff77288e91
  Stored in directory: /root/.cache/pip/wheels/bf/d4/ec/fd4f622fcbefb7521f149905295b2c26adecb23af38aa28217
Successfully built implicit


**Import libraries, Read the data¶**


In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import time
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD, SlopeOne, NMF, SVDpp, KNNBaseline, CoClustering, accuracy
from surprise.model_selection import cross_validate
from __future__ import (absolute_import, division, print_function,unicode_literals)
from sklearn.model_selection import train_test_split
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import sys
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random
from sklearn.preprocessing import MinMaxScaler
import implicit


In [5]:
from google.colab import files
uploaded = files.upload()

Saving hotel_data_final.csv to hotel_data_final.csv


In [6]:
import io
df = pd.read_csv(io.BytesIO(uploaded['hotel_data_final.csv']))

In [7]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [8]:
# Explore Data
type(df)
df.size
df.shape
df.head()
df.tail()

pandas.core.frame.DataFrame

280728

(23394, 12)

,Hotel_id,Hotel,Address,City,Title,Users,Reviews,Rating,Hotel_Type,Price,Hotel_Category,User_id
0,0,La Quinta,5820 Walden Rd,Beaumont,This hotel is a dump! Upon ch...,margierodriguez.mcqueen,This hotel is a dump! Upon checking in was tol...,1.0,2.0,Rs.8989,"Hotels and Motels,Lodging,Meeting & Event Plan...",15400
1,1,The Point,Po Box 1327,Saranac Lake,Great family time,motomomMaryland,The point is possibly one of our favorite rest...,5.0,NaN,NaN,"Hotels,Restaurants",15740
2,1,The Point,Po Box 1327,Saranac Lake,Throw back to the Gatsby Era - don't miss The ...,ellen00007,From the minute you arrive until the minute yo...,5.0,NaN,NaN,"Hotels,Restaurants",13760
3,1,The Point,Po Box 1327,Saranac Lake,The Point of it all,Szerzek,That is the name on one of their boats. And th...,5.0,NaN,NaN,"Hotels,Restaurants",11027
4,1,The Point,Po Box 1327,Saranac Lake,The most amazing week end!,Sarahbnyc83,I was lucky enough to spend 2 nights at The Po...,5.0,NaN,NaN,"Hotels,Restaurants",10306


,Hotel_id,Hotel,Address,City,Title,Users,Reviews,Rating,Hotel_Type,Price,Hotel_Category,User_id
23389,3128,Polynesian Plaza,2131 Kalakaua Ave,Honolulu,IN SEVERE NEED OF UPDATE - OVERHAUL,fordpipefitter,We spent 2 weeks in this hotel due to the cost...,2.0,2.0,Rs.17718,"Vacation Rentals,Resorts & Vacation Cottages,V...",13911
23390,3129,Quality Inn,2120 Jameson Pl S W,Decatur,NaN,RABBITS,VERY FRIENDLY AND GREAT PRICE. I HAVE STAYED A...,5.0,2.0,Rs.7373,"Hotels and Motels,Lodging",9306
23391,3129,Quality Inn,2120 Jameson Pl S W,Decatur,NaN,Contributor,Editorial Review by Citysearch Editors Colonia...,5.0,2.0,Rs.7373,"Hotels and Motels,Lodging",2888
23392,3130,Quality Inn,12439 Northwest Fwy,Houston,NaN,Houstontraveler,What Happened to this Place!!!! I come this ho...,NaN,2.0,Rs.7373,"Hotel & Motel Reservations,Hotels and Motels",4949
23393,3130,Quality Inn,12439 Northwest Fwy,Houston,NaN,Robyn Galbos,Budget-minded travelers appreciate amenities l...,NaN,2.0,Rs.7373,"Hotel & Motel Reservations,Hotels and Motels",9807


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23394 entries, 0 to 23393
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Hotel_id        23394 non-null  int64  
 1   Hotel           23394 non-null  object 
 2   Address         23394 non-null  object 
 3   City            23394 non-null  object 
 4   Title           23342 non-null  object 
 5   Users           23394 non-null  object 
 6   Reviews         23393 non-null  object 
 7   Rating          23365 non-null  float64
 8   Hotel_Type      22432 non-null  float64
 9   Price           21881 non-null  object 
 10  Hotel_Category  23394 non-null  object 
 11  User_id         23394 non-null  int64  
dtypes: float64(2), int64(2), object(8)
memory usage: 2.1+ MB


In [10]:
# Checking for missing values
pd.isnull(df).any()
pd.isnull(df).sum()

Hotel_id          False
Hotel             False
Address           False
City              False
Title              True
Users             False
Reviews            True
Rating             True
Hotel_Type         True
Price              True
Hotel_Category    False
User_id           False
dtype: bool

Hotel_id             0
Hotel                0
Address              0
City                 0
Title               52
Users                0
Reviews              1
Rating              29
Hotel_Type         962
Price             1513
Hotel_Category       0
User_id              0
dtype: int64

In [11]:
# Dropping the missing values
df = df.dropna()
df = df.reset_index(drop=True)
print('Shape of dataset ',df.shape)
print(df.columns)

Shape of dataset  (21770, 12)
Index(['Hotel_id', 'Hotel', 'Address', 'City', 'Title', 'Users', 'Reviews',
       'Rating', 'Hotel_Type', 'Price', 'Hotel_Category', 'User_id'],
      dtype='object')


**Train-Test Split**

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(df[['User_id','Hotel_id']],df['Rating'],test_size=0.25, 
                                                  random_state=0)

y_train=pd.DataFrame(y_train)
train=X_train.join(y_train)

y_test=pd.DataFrame(y_test)
test=X_test.join(y_test)

In [13]:
X_train.shape
X_test.shape
y_train.shape
y_test.shape

(16327, 2)

(5443, 2)

(16327, 1)

(5443, 1)

**Collaborative Filtering**

Collaborative Filtering is based on the idea that users similar to a me can be used to predict how much I will like a particular product or service those users have used/experienced but I have not.

**1- Simple Collaborative filtering base on mean**

Recommend based on other user's rating histories.

In [14]:
def collab_mean(user_id, hotel_id):
    user_condition = X_train.Hotel_id == hotel_id
    return y_train[user_condition].mean()

i=30
j=96
print('Collaborative mean filter for userId:',i, 'and hotelid:', j, 'is:',collab_mean(i,j).values)
print('Actual rating value is:', df[(df.User_id==i)&(df.Hotel_id==j)]['Rating'].mean())
print('')

i=3529
j=25
print('Collaborative mean filter for userId:',i, 'and hotelid:', j, 'is:',collab_mean(i,j).values)
print('Actual rating value is:', df[(df.User_id==i)&(df.Hotel_id==j)]['Rating'].mean())
print('')



i=13532
j=9
print('Collaborative mean filter for userId:',i, 'and hotelid:', j, 'is:',collab_mean(i,j).values)
print('Actual rating value is:', df[(df.User_id==i)&(df.Hotel_id==j)]['Rating'].mean())
print('')

Collaborative mean filter for userId: 30 and hotelid: 96 is: [4.18181818]
Actual rating value is: 5.0

Collaborative mean filter for userId: 3529 and hotelid: 25 is: [4.20930233]
Actual rating value is: 4.0

Collaborative mean filter for userId: 13532 and hotelid: 9 is: [4.20430108]
Actual rating value is: 4.0



**2- Matrix Factorization-based algorithms**

The idea is basically to take a large (or potentially huge) matrix and factor it into some smaller representation of the original matrix. You can think of it in the same way as we would take a large number and factor it into two much smaller primes. We end up with two or more lower dimensional matrices whose product equals the original one. When we talk about collaborative filtering for recommender systems we want to solve the problem of our original matrix having millions of different dimensions, but our “tastes” not being nearly as complex. Even if i’ve viewed hundreds of items they might just express a couple of different tastes. Here we can actually use matrix factorization to mathematically reduce the dimensionality of our original “all users by all items” matrix into something much smaller that represents “all items by some taste dimensions” and “all users by some taste dimensions”. These dimensions are called latent or hidden features and we learn them from our data. Doing this reduction and working with fewer dimensions makes it both much more computationally efficient and but also gives us better results since we can reason about items in this more compact “taste space”. If we can express each user as a vector of their taste values, and at the same time express each item as a vector of what tastes they represent. You can see we can quite easily make a recommendation. This also gives us the ability to find connections between users who have no specific items in common but share common tastes.

**SVD**

SVD is a matrix factorization technique that is usually used to reduce the number of features of a data set by reducing space dimensions from N to K where K < N. For the purpose of the recommendation systems however, we are only interested in the matrix factorization part keeping same dimensionality. The matrix factorization is done on the user-item ratings matrix.

I have used Surprise library which has several powerful algorithms like Singular Value Decomposition (SVD), Non-negative Matrix Factorization (NMF), K Nearest Neighbor (KNN), and CoClusternig to minimise RMSE (Root Mean Square Error) and give recommendations.

In [15]:
data = Dataset.load_from_df(df[['User_id', 'Hotel_id', 'Rating']], Reader())


In [16]:
Models=[SVD(), NMF(), SlopeOne(), KNNBaseline(), CoClustering()]

for i in Models:
    start = time.time()
    cross_validate(i, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
    end = time.time()
    print('Total Elapsed time with model is:',end - start)
    print(' ')
    print(' ')

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0782  1.1069  1.0607  1.0629  1.0717  1.0761  0.0166  
MAE (testset)     0.8350  0.8553  0.8236  0.8313  0.8348  0.8360  0.0105  
Fit time          0.96    0.95    0.95    0.96    0.96    0.96    0.00    
Test time         0.10    0.02    0.02    0.02    0.02    0.04    0.03    


{'fit_time': (0.9581828117370605,
  0.9506053924560547,
  0.9534204006195068,
  0.9585602283477783,
  0.9617354869842529),
 'test_mae': array([0.83502231, 0.85528282, 0.8236316 , 0.83133511, 0.8348427 ]),
 'test_rmse': array([1.0781598 , 1.10690976, 1.06065746, 1.06293639, 1.0717246 ]),
 'test_time': (0.09820270538330078,
  0.021178245544433594,
  0.02171778678894043,
  0.021506071090698242,
  0.020934581756591797)}

Total Elapsed time with model is: 5.1907594203948975
 
 
Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1979  1.2050  1.2201  1.2086  1.2166  1.2097  0.0080  
MAE (testset)     0.9218  0.9427  0.9558  0.9416  0.9418  0.9408  0.0109  
Fit time          1.71    1.71    1.73    1.72    1.70    1.71    0.01    
Test time         0.02    0.02    0.02    0.02    0.02    0.02    0.00    


{'fit_time': (1.712583303451538,
  1.7123665809631348,
  1.730083703994751,
  1.7189643383026123,
  1.6955337524414062),
 'test_mae': array([0.92182737, 0.94274674, 0.95582021, 0.94158454, 0.9418012 ]),
 'test_rmse': array([1.19790562, 1.20503334, 1.22014944, 1.20863953, 1.21655955]),
 'test_time': (0.0215301513671875,
  0.021588802337646484,
  0.02376270294189453,
  0.02187800407409668,
  0.021091222763061523)}

Total Elapsed time with model is: 8.97967004776001
 
 
Evaluating RMSE, MAE of algorithm SlopeOne on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2026  1.2323  1.2177  1.2008  1.2109  1.2129  0.0115  
MAE (testset)     0.9202  0.9259  0.9273  0.9132  0.9233  0.9220  0.0050  
Fit time          0.23    0.24    0.21    0.21    0.24    0.22    0.01    
Test time         0.04    0.04    0.04    0.04    0.04    0.04    0.00    


...

Total Elapsed time with model is: 1.545750379562378
 
 
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0885  1.0756  1.0792  1.1071  1.0857  1.0872  0.0109  
MAE (testset)     0.8468  0.8365  0.8381  0.8495  0.8405  0.8423  0.0050  
Fit time          6.57    5.86    5.63    6.53    5.54    6.02    0.44    
Test time         0.10    0.09    0.10    0.10    0.09    0.10

{'fit_time': (6.568473815917969,
  5.859039068222046,
  5.625234127044678,
  6.525378942489624,
  5.535629987716675),
 'test_mae': array([0.84682128, 0.83646407, 0.83810339, 0.84948785, 0.84048743]),
 'test_rmse': array([1.08852186, 1.07555529, 1.07920855, 1.10708831, 1.08572116]),
 'test_time': (0.09599471092224121,
  0.09494137763977051,
  0.0950920581817627,
  0.10105466842651367,
  0.09331393241882324)}

Total Elapsed time with model is: 30.92216444015503
 
 
Evaluating RMSE, MAE of algorithm CoClustering on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1680  1.2148  1.2260  1.1752  1.1873  1.1943  0.0225  
MAE (testset)     0.9066  0.9271  0.9393  0.8968  0.9108  0.9161  0.0152  
Fit time          1.31    1.28    1.29    1.31    1.28    1.30    0.01    
Test time         0.02    0.02    0.02    0.02    0.02    0.02    0.00    


{'fit_time': (1.309453010559082,
  1.2844417095184326,
  1.2912521362304688,
  1.3104567527770996,
  1.2843718528747559),
 'test_mae': array([0.90664148, 0.92706454, 0.93933833, 0.89683381, 0.91077642]),
 'test_rmse': array([1.16797688, 1.21482458, 1.22601991, 1.17516431, 1.18727847]),
 'test_time': (0.018099308013916016,
  0.018281936645507812,
  0.01789569854736328,
  0.01903247833251953,
  0.017935514450073242)}

Total Elapsed time with model is: 6.824107885360718
 
 


**Comparing the predicted rating values of some samples with their actual values**

In [17]:
from surprise.model_selection import train_test_split
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

Models=[SVD(), NMF(), SlopeOne(), KNNBaseline(), CoClustering()]
m=['SVD','NMF','Slop One','KNN Baseline', 'CoClustering']

count=0

for algo in Models:
    print(m[count])
    start = time.time()
    algo.fit(trainset)
    predictions = algo.test(testset)
    end = time.time()    
    print('Total Elapsed time with model is:',round(end - start,3)) 
    i=30
    j=96
    print('Prediction of a particular user:', algo.predict(i,j, 3).est)
    print('Actual rating value is:', df[(df.User_id==i)&(df.Hotel_id==j)]['Rating'].mean())
    print('')
    i=3529
    j=25
    print('Prediction of a particular user:', algo.predict(i,j, 3).est)
    print('Actual rating value is:', df[(df.User_id==i)&(df.Hotel_id==j)]['Rating'].mean())
    print('')
    i=13532
    j=9
    print('Prediction of a particular user:', algo.predict(i,j, 3).est)
    print('Actual rating value is:', df[(df.User_id==i)&(df.Hotel_id==j)]['Rating'].mean())
    print('')
    print('')
    count+=1

SVD


Total Elapsed time with model is: 0.955
Prediction of a particular user: 4.286017891468428
Actual rating value is: 5.0

Prediction of a particular user: 4.10336259464257
Actual rating value is: 4.0

Prediction of a particular user: 4.055758815311502
Actual rating value is: 4.0


NMF


Total Elapsed time with model is: 1.684
Prediction of a particular user: 4.896922967469765
Actual rating value is: 5.0

Prediction of a particular user: 3.6828028773477195
Actual rating value is: 4.0

Prediction of a particular user: 3.9313629456937025
Actual rating value is: 4.0


Slop One


Total Elapsed time with model is: 0.251
Prediction of a particular user: 5
Actual rating value is: 5.0

Prediction of a particular user: 3.606837606837607
Actual rating value is: 4.0

Prediction of a particular user: 4.0
Actual rating value is: 4.0


KNN Baseline
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


Total Elapsed time with model is: 4.971
Prediction of a particular user: 4.994851594864499
Actual rating value is: 5.0

Prediction of a particular user: 4.172558808838583
Actual rating value is: 4.0

Prediction of a particular user: 4.014300280182937
Actual rating value is: 4.0


CoClustering


Total Elapsed time with model is: 1.256
Prediction of a particular user: 5
Actual rating value is: 5.0

Prediction of a particular user: 2.0998074250596535
Actual rating value is: 4.0

Prediction of a particular user: 4.172034148342725
Actual rating value is: 4.0




**3- Alternating Least Squares (ALS) Collaborative Filtering**

Alternating Least Squares (ALS) is a the model we’ll use to fit our data and find similarities. ALS is an iterative optimization process where we for every iteration try to arrive closer and closer to a factorized representation of our original data. We have our original matrix R of size u x i with our users, items and some type of feedback data. We then want to find a way to turn that into one matrix with users and hidden features of size u x f and one with items and hidden features of size f x i. In U and V we have weights for how each user/item relates to each feature. What we do is we calculate U and V so that their product approximates R as closely as possible: R ≈ U x V.

By randomly assigning the values in U and V and using least squares iteratively we can arrive at what weights yield the best approximation of R. The least squares approach in it’s basic forms means fitting some line to the data, measuring the sum of squared distances from all points to the line and trying to get an optimal fit by minimising this value.

With the alternating least squares approach we use the same idea but iteratively alternate between optimizing U and fixing V and vice versa. We do this for each iteration to arrive closer to R = U x V.

In [18]:
data=df
sparse_item_user = sparse.csr_matrix((data['Rating'].astype(float), (data['Hotel_id'], data['User_id'])))
sparse_user_item = sparse.csr_matrix((data['Rating'].astype(float), (data['Hotel_id'], data['User_id'])))

# Initialize the als model and fit it using the sparse item-user matrix
model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=20)

# Calculate the confidence by multiplying it by our alpha value.
alpha_val = 15
data_conf = (sparse_item_user * alpha_val).astype('double')

model.fit(data_conf)

GPU training requires factor size to be a multiple of 32. Increasing factors from 20 to 32.


**Finding Hotels Similar to Best Western Lamplighter Inn Suites at SDSU:**

In [19]:
#---------------------
# FIND SIMILAR HOTELS
#---------------------
Hotel_id = 318 #Best Western Lamplighter Inn Suites at SDSU
n_similar = 10

# Use implicit to get similar items.
similar = model.similar_items(Hotel_id, n_similar)

a=[]
b=[]
c=[]
# Print the names of our most similar movies
for item in similar:
    idx, score = item
    a.append(data.Hotel_id.loc[data.Hotel_id == idx].iloc[0])
    b.append(score)
    recommendations = pd.DataFrame({'HotelId': a, 'Score': b})

for i in a:
    c.append(df[df.Hotel_id==i].Hotel.iloc[0])
recommendations['Hotel']=c
recommendations

,HotelId,Score,Hotel
0,318,1.000000,Best Western Lamplighter Inn Suites at SDSU
1,1049,0.999451,Hampton Inn Myrtle Beach-Northwood
2,1045,0.854283,Courtyard Macon
3,2292,0.829147,Home2 Suites By Hilton Greenville Airport
4,2894,0.770329,Americas Best Value Inn
5,1324,0.720670,"The Westin Poinsett, Greenville"
6,2425,0.605194,W Austin
7,1216,0.588287,Ramada Costa Mesa/Newport Beach
8,2771,0.578541,Hampton Inn Suites Indianapolis-Keystone
9,431,0.554224,Hilton Garden Inn Atlanta Midtown


**Finding Hotels Similar to Nob Hill Hotel:**

In [20]:
#---------------------
# FIND SIMILAR HOTELS
#---------------------
Hotel_id = 296 #Nob Hill Hotel
n_similar = 10

# Use implicit to get similar items.
similar = model.similar_items(Hotel_id, n_similar)

a=[]
b=[]
c=[]
# Print the names of our most similar movies
for item in similar:
    idx, score = item
    a.append(data.Hotel_id.loc[data.Hotel_id == idx].iloc[0])
    b.append(score)
    recommendations = pd.DataFrame({'HotelId': a, 'Score': b})

for i in a:
    c.append(df[df.Hotel_id==i].Hotel.iloc[0])

recommendations['Hotel']=c
recommendations

,HotelId,Score,Hotel
0,296,1.000000,Nob Hill Hotel
1,713,0.927863,Barn Motor Inn
2,1291,0.927658,Magnuson Grand Hotel
3,2400,0.591689,Red Roof Inn
4,1942,0.589663,Comfort Suites-independence
5,369,0.589618,Comfort Inn Suites
6,519,0.589613,Mountain View Inn
7,3061,0.575069,Sheraton Grand Bangalore Hotel at Brigade Gateway
8,872,0.565855,Americas Best Value Inn
9,1922,0.565839,Best Western Lake Okeechobee


**Recommend Hotels to User_id = 20:**

In [21]:
#------------------------------
# CREATE USER RECOMMENDATIONS
#------------------------------

# Create recommendations for user with id 20
User_id =20

# Use the implicit recommender.
recommended = model.recommend(User_id, sparse_user_item)

hotels = []
scores = []
c=[]

for item in recommended:
    idx, score=item
    hotels.append(data.Hotel_id.loc[data.Hotel_id==idx].iloc[0])
    scores.append(score)


recommendations = pd.DataFrame({'HotelId': hotels, 'Score': scores})

for i in hotels:
    c.append(df[df.Hotel_id==i].Hotel.iloc[0])
recommendations['Hotel']=c
recommendations

,HotelId,Score,Hotel
0,9,0.947020,Doubletree By Hilton West Palm Beach Airport
1,38,0.264315,Doubletree By Hilton Hotel Bay City - Riverfront
2,2822,0.260166,French Market Inn
3,14,0.258840,Staybridge Suites Tyler University Area
4,1283,0.228071,Hampton Inn & Suites By Hilton Miami/Brickell-...
5,620,0.213321,SpringHill Suites Seattle Downtown/South Lake ...
6,3001,0.207065,Waikiki Resort Hotel
7,3025,0.204718,Breeze Suites
8,294,0.203978,Greenwich Inn
9,5,0.200620,Country Inn and Suites By Carlson Corbin


**Recommend Hotels to User_id = 995:**

In [22]:
#------------------------------
# CREATE USER RECOMMENDATIONS
#------------------------------

# Create recommendations for user with id 995
User_id =995

# Use the implicit recommender.
recommended = model.recommend(User_id, sparse_user_item)

hotels = []
scores = []
c=[]

for item in recommended:
    idx, score=item
    hotels.append(data.Hotel_id.loc[data.Hotel_id==idx].iloc[0])
    scores.append(score)


recommendations = pd.DataFrame({'HotelId': hotels, 'Score': scores})

for i in hotels:
    c.append(df[df.Hotel_id==i].Hotel.iloc[0])
    
recommendations['Hotel']=c
recommendations

,HotelId,Score,Hotel
0,3113,0.969817,Best Western Plus Waterville Grand Hotel
1,2,0.899147,Inn At Queen Anne
2,85,0.705014,Homewood Suites by Hilton Baltimore
3,25,0.697006,Best Western Plus Arlington North Hotel and Su...
4,46,0.640948,Howard Johnson Inn Columbia
5,42,0.636532,Super 8 Ithaca
6,6,0.624031,Ambassadors Inn and Suites
7,2189,0.619027,Metro Points Hotel-Washington North
8,2741,0.615025,Best Western Springfield
9,3118,0.586928,Fairfield Inn By Marriott Binghamton
